# Generation of Simulation Data

### imports and global parameters

In [ ]:
# Imports
import numpy as np
import os
from multiprocessing import Pool
import time

# Local Imports
from simple_pendulum.model.pendulum_plant import PendulumPlant
from simple_pendulum.simulation.simulation import Simulator
from simple_pendulum.controllers.lqr.lqr_controller import LQRController

In [ ]:
# Simulation Parameters
N = 500
dt = 0.01
t_final = 10.0
boundaries = [0,2*np.pi,-10,10]

# Standard Pendulum Parameters
mass = 0.676
length = 0.45
torque_limit = 2
damping = 0.1
coulomb_fric = 0.0
gravity = 9.81
inertia = mass*length*length

### functions for generating trajectories from random initial values and analasys of those

In [ ]:
# Generate Random Points
def getPoints(N, boundaries, seed):
    np.random.seed(seed)
    points = np.zeros((N,2))
    for i in range(N):
        state = np.array([np.random.uniform(boundaries[0], boundaries[1]), np.random.uniform(boundaries[2], boundaries[3])])
        points[i,0] = state[0]
        points[i,1] = state[1]
    return points


# Saving Parameters and Data as .csv-file
# to read use np.genfromtxt("path/to/file.csv", comments="#", delimiter=',')
def save_data(sim_data, filepath):
    T,X,U = sim_data
    T,X,U = np.array(T,ndmin=2).transpose(),np.array(X),np.array(U,ndmin=2).transpose()
    out = np.concatenate((T,X,U), axis = 1)
    header='Time [s], angle [rad], angular velocity [rad/s], torque [Nm]'
    np.savetxt(filepath, out, delimiter=',', header=header)

def save_params(params, filepath):
    header='mass, length, torque limit, damping, coulomb friction, gravity, Q_11, Q_22, R, N, seed'
    np.savetxt(filepath, params, delimiter=',', header=header) 
    
def save_table(table, filepath):
    
    header='initial angle, initial angular velocity, converges to upright, violates torque limit'
    np.savetxt(filepath, table, delimiter=',', header=header) 
    
# Generate Simulation Data
def gen_data(name_of_run, mass, length, torque_limit, seed):
    
    # Pendulum Parameters
    damping = 0.1
    coulomb_fric = 0.0
    gravity = 9.81
    
    # Optimazation Parameters
    Q_11 = 1
    Q_22 = 1
    R = 1
    
    # Derived Inertia
    inertia = mass*length*length

    # Initializing Simulation
    pendulum = PendulumPlant(mass=mass,
                             length=length,
                             damping=damping,
                             gravity=gravity,
                             coulomb_fric=coulomb_fric,
                             inertia=inertia,
                             torque_limit=torque_limit)

    capped = LQRController(mass=mass,
                           length=length,
                           damping=damping,
                           gravity=gravity,
                           torque_limit=torque_limit,
                           Q = np.diag((Q_11,Q_22)),
                           R = np.array([[R]]))
    capped.set_clip()
    
    free = LQRController(mass=mass,
                         length=length,
                         damping=damping,
                         gravity=gravity,
                         torque_limit=np.inf,
                         Q = np.diag((Q_11,Q_22)),
                         R = np.array([[R]]))
                         
                         
    sim = Simulator(plant=pendulum)
    
    # Initialize Truth Table
    table = np.zeros((N,4)) #[theta_0, omega_0, converges to upright, violates torque limit]
    
    # saving Parameters
    params = [[mass,
               length,
               torque_limit,
               damping,
               coulomb_fric,
               gravity,
               Q_11,
               Q_22,
               R,
               N,
               seed]]
    path = "sim_data/full_data/" + name_of_run + "/"
    path2 = "sim_data/parameters_and_tables/" + name_of_run + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(path2):
        os.makedirs(path2)
    save_params(params, path + "parameters.csv")
    save_params(params, path2 + "parameters.csv")
    
    # Running Simulation
    points = getPoints(N, boundaries, seed)
    for i in range(len(points)):
        T, X, U = sim.simulate(t0=0.0,
                               x0=points[i,:],
                               tf=t_final,
                               dt=dt,
                               controller=capped,
                               integrator="runge_kutta")
        T = np.concatenate((np.array(0, ndmin=1),T))
        X = np.concatenate((np.array(points[i,:], ndmin = 2),X))
        save_data((T[:-1],X[:-1],U), path + format(i, '05d') + ".csv")
        
        # Filling Truth Table
        table[i,0] = points[i,:][0]
        table[i,1] = points[i,:][1]
        fin = np.array([X[-1][0]%(np.pi*2)-np.pi, X[-1][1]])
        revolved = False
        violated = False
        for j in range(0, len(X)):
            if X[j][0] < 0 or X[j][0] > 2*np.pi:
                revolved = True
            if np.abs(free.get_control_output(X[j][0],X[j][1])[2]) > torque_limit:
                violated = True
        # stabilizes
        if not revolved and (np.abs(fin) < 10**(-5)).all():
            table[i,2] = True
        # violates
        if violated:
            table[i,3] = True
        
    save_table(table, path + "table.csv")
    save_table(table, path2 + "table.csv")
        
def gen_data_packed(package):
    name_of_run, mass, length, torque_limit, seed = package
    gen_data(name_of_run, mass, length, torque_limit, seed)
    

### numerical experiments for different architectures and torque limitations

In [ ]:
gamma_short = 10
gamma_long = .1

m_normal = mass
l_normal = length
mlg = m_normal*l_normal*gravity
m_long = np.sqrt(mlg*gamma_long/gravity)
l_long = m_long/gamma_long
m_short = np.sqrt(mlg*gamma_short/gravity)
l_short = m_short/gamma_short


pool = Pool()
args = [["normal_1", m_normal, l_normal, mlg, np.random.randint(10000)],
        ["normal_2", m_normal, l_normal, mlg/2, np.random.randint(10000)],
        ["normal_4", m_normal, l_normal, mlg/4, np.random.randint(10000)],
        ["normal_8", m_normal, l_normal, mlg/8, np.random.randint(10000)],
        ["long_1", m_long, l_long, mlg, np.random.randint(10000)],
        ["long_2", m_long, l_long, mlg/2, np.random.randint(10000)],
        ["long_4", m_long, l_long, mlg/4, np.random.randint(10000)],
        ["long_8", m_long, l_long, mlg/8, np.random.randint(10000)],
        ["short_1", m_short, l_short, mlg, np.random.randint(10000)],
        ["short_2", m_short, l_short, mlg/2, np.random.randint(10000)],
        ["short_4", m_short, l_short, mlg/4, np.random.randint(10000)],
        ["short_8", m_short, l_short, mlg/8, np.random.randint(10000)]
        ]
t = time.time()
pool.map(gen_data_packed, args)
print(time.time()-t)

Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-11:
Process ForkPoolWorker-6:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-16:
Process ForkPoolWorker-1:
Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multipr